In [6]:
import pandas as pd, re
import numpy as np
import getschools as gs

In [7]:
def get_df(group):
    ret = pd.read_csv("../scripts/noequals/CTSchoolDaySAT-"+group+".csv",skiprows=2,header=None)
    ret = gs.merge_01(ret)
    ret = gs.clean_colnames(ret)
    
    ret["district_code"] = ret["district_code"].str.replace("\"","")
    ret["school_code"] = ret["school_code"].str.replace("\"","")
    
    ret["year"] = "2015-16"
    
    ret = gs.add_drg(ret)
    ret["average_score"] = ret["level_3_4met_or_exceeded_averagescore"]
    return ret

get_df("all")

,district,district_code,school,school_code,subject,total_numberwith_scored_tests,level_1not_met_count,level_1not_met_pct,level_2approaching_count,level_2approaching_pct,level_3met_count,level_3met_pct,level_4exceeded_count,level_4exceeded_pct,level_3_4met_or_exceeded_count,level_3_4met_or_exceeded_pct,level_3_4met_or_exceeded_averagescore,year,drg,average_score
2,Achievement First Hartford Academy Inc. District,2880013,Achievement First Hartford Academy Inc.,2880113,ELA,30,*,*,7,23.3,17,56.7,*,*,*,*,529,2015-16,None,529
3,Achievement First Hartford Academy Inc. District,2880013,Achievement First Hartford Academy Inc.,2880113,Math,30,*,*,13,43.3,14,46.7,*,*,*,*,537,2015-16,None,537
4,Amistad Academy District,2790013,Amistad Academy,2795113,ELA,44,6,13.6,*,*,28,63.6,*,*,*,*,515,2015-16,None,515
5,Amistad Academy District,2790013,Amistad Academy,2795113,Math,44,11,25.0,23,52.3,*,*,*,*,10,22.7,470,2015-16,None,470
6,Ansonia School District,0020011,Ansonia High School,0026111,ELA,*,*,*,*,*,*,*,*,*,*,*,*,2015-16,H,*
7,Ansonia School District,0020011,Ansonia High School,0026111,Math,*,*,*,*,*,*,*,*,*,*,*,*,2015-16,H,*
8,Ansonia School District,0020011,P.A.C.E. (Positive And Creative Education),0020111,ELA,*,*,*,*,*,*,*,*,*,*,*,*,2015-16,H,*
9,Ansonia School District,0020011,P.A.C.E. (Positive And Creative Education),0020111,Math,*,*,*,*,*,*,*,*,*,*,*,*,2015-16,H,*
10,Area Cooperative Educational Services,2440014,Whitney High School North,2449114,ELA,18,*,*,*,*,*,*,*,*,*,*,*,2015-16,None,*
11,Area Cooperative Educational Services,2440014,Whitney High School North,2449114,Math,16,*,*,*,*,*,*,*,*,*,*,*,2015-16,None,*


In [8]:
get_df("all").to_csv("../clean/for_db/sat_all.csv",index=False)

In [9]:
get_df("race-eth").to_csv("../clean/for_db/sat_race.csv",index=False)

In [10]:
def weighted_avg(df, groups=[]):
    ret = df.copy()
    
    ret["level_3_4met_or_exceeded_averagescore"] = pd.to_numeric(ret["level_3_4met_or_exceeded_averagescore"],errors="coerce")
    ret["total_numberwith_scored_tests"] = pd.to_numeric(ret["total_numberwith_scored_tests"],errors="coerce")
    ret["weight"] = ret["level_3_4met_or_exceeded_averagescore"] * ret["total_numberwith_scored_tests"]
    
    
    ret = ret[ret["total_numberwith_scored_tests"].notnull()]
    ret = ret[ret["level_3_4met_or_exceeded_averagescore"].notnull()]

    ret = ret.groupby(["year","subject","drg"] + groups).agg({
            "weight":np.sum,
            "total_numberwith_scored_tests":np.sum
        }).reset_index()
    
    ret["average_score"] = ret["weight"] / ret["total_numberwith_scored_tests"]
    return ret
    
weighted_avg(get_df("all"))

,year,subject,drg,total_numberwith_scored_tests,weight,average_score
0,2015-16,ELA,A,1609.0,968851.0,602.144810
1,2015-16,ELA,B,5937.0,3401549.0,572.940711
2,2015-16,ELA,C,2124.0,1189849.0,560.192561
3,2015-16,ELA,D,4464.0,2384534.0,534.169803
4,2015-16,ELA,E,1233.0,661508.0,536.502839
5,2015-16,ELA,F,778.0,396282.0,509.359897
6,2015-16,ELA,G,3148.0,1591720.0,505.628971
7,2015-16,ELA,H,3752.0,1820391.0,485.178838
8,2015-16,ELA,I,4889.0,2182723.0,446.455921
9,2015-16,Math,A,1609.0,965828.0,600.266004


In [11]:
weighted_avg(get_df("all")).to_csv("../clean/for_db/drg_sat_all.csv", index=False)

In [12]:
weighted_avg(get_df("race-eth"),groups=["race"]).to_csv("../clean/for_db/drg_sat_race.csv", index=False)

In [13]:
weighted_avg(get_df("ell"),groups=["ell"])#.to_csv("../clean/for_db/drg_sat_race.csv", index=False)

,year,subject,drg,ell,total_numberwith_scored_tests,weight,average_score
0,2015-16,ELA,A,N,1252.0,756616.0,604.325879
1,2015-16,ELA,B,N,3809.0,2184078.0,573.399317
2,2015-16,ELA,C,N,1437.0,804830.0,560.076548
3,2015-16,ELA,D,N,1460.0,777289.0,532.389726
4,2015-16,ELA,E,N,993.0,535460.0,539.234642
5,2015-16,ELA,F,N,853.0,437075.0,512.397421
6,2015-16,ELA,G,N,3252.0,1642589.0,505.101169
7,2015-16,ELA,G,Y,44.0,16940.0,385.000000
8,2015-16,ELA,H,N,3186.0,1580745.0,496.153484
9,2015-16,ELA,H,Y,334.0,128547.0,384.871257


In [14]:
def state_report(group="all"):
    
    ret = pd.read_csv("../scripts/noequals/CTSchoolDaySAT-state-"+group+".csv",skiprows=2,header=None)
    ret = gs.merge_01(ret)
    ret = gs.clean_colnames(ret)
    
    ret["level"] = "Statewide"
    ret["year"] = "2015-16"
    
    ret["average_score"] = ret["level_3_4met_or_exceeded_averagescore"]
    
    return ret

state_report("race")

,district,subject,race,total_numberwith_scored_tests,level_1not_met_count,level_1not_met_pct,level_2approaching_count,level_2approaching_pct,level_3met_count,level_3met_pct,level_4exceeded_count,level_4exceeded_pct,level_3_4met_or_exceeded_count,level_3_4met_or_exceeded_pct,level_3_4met_or_exceeded_averagescore,level,year,average_score
2,State of Connecticut,ELA,American Indian or Alaska Native,104,*,*,23,22.1,51,49.0,*,*,*,*,507,Statewide,2015-16,507
3,State of Connecticut,ELA,Asian,1687,155,9.2,201,11.9,800,47.4,531,31.5,1331,78.9,565,Statewide,2015-16,565
4,State of Connecticut,ELA,Black or African American,4666,1707,36.6,1261,27.0,1553,33.3,145,3.1,1698,36.4,451,Statewide,2015-16,451
5,State of Connecticut,ELA,Hispanic/Latino of any race,6970,2525,36.2,1729,24.8,2406,34.5,310,4.4,2716,39.0,457,Statewide,2015-16,457
6,State of Connecticut,ELA,Native Hawaiian or Other Pacific Islander,31,9,29.0,*,*,12,38.7,*,*,*,*,491,Statewide,2015-16,491
7,State of Connecticut,ELA,Two or More Races,755,126,16.7,148,19.6,367,48.6,114,15.1,481,63.7,517,Statewide,2015-16,517
8,State of Connecticut,ELA,White,23640,2145,9.1,3187,13.5,12896,54.6,5412,22.9,18308,77.4,548,Statewide,2015-16,548
9,State of Connecticut,Math,American Indian or Alaska Native,104,25,24.0,50,48.1,*,*,*,*,29,27.9,487,Statewide,2015-16,487
10,State of Connecticut,Math,Asian,1686,141,8.4,412,24.4,596,35.3,537,31.9,1133,67.2,582,Statewide,2015-16,582
11,State of Connecticut,Math,Black or African American,4660,2068,44.4,2016,43.3,521,11.2,55,1.2,576,12.4,431,Statewide,2015-16,431


In [15]:
state_report().to_csv("../clean/for_db/state_sat_all.csv",index=False)

In [16]:
state_report("race").to_csv("../clean/for_db/state_sat_race.csv",index=False)

In [17]:
state_report("ell").to_csv("../clean/for_db/state_sat_ell.csv",index=False)